* **Business /research question?**
    * A new Learning System will be launched. Which courses are worth it to migrate.
    * Criterias
        * TBD
        * Course completion rate in ration to useres by local over a timeframe
* **Where do I get the data from?**
     * Reports from the Learning Record Store. 
* **Potential hurdles here?**
     * Course creators are not consistent with providing details for a course
     * Time_to_complete format
     * Deleted users
* **What is my MVP - minimum viable product?**
     * Analysis of one Course Category 
* **Current state of work**
     * Establish a SQL Database + normalization
     * Understanding the features
     * Cleaning course table

In [42]:
# Load Base libraries
import pandas as pd
import numpy as np
import math
import datetime
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
pd.set_option('display.max_columns', None)

In [43]:
# Access mysql database
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

 ···········


In [44]:
# establish conncetion and engine
connection_string = 'mysql+pymysql://root:' + password + '@localhost/clz'
engine = create_engine(connection_string)

In [45]:
# load data and assign to variable
df_course = pd.read_sql_query('SELECT * FROM course', engine)

In [46]:
df_course.shape

(1150, 13)

In [48]:
# lower case header names
cols = []
for i in range(len(df_course.columns)):
    cols.append(df_course.columns[i].lower())
df_course.columns = cols

In [49]:
df_course.columns

Index(['course_id', 'course_name', 'course_language', 'location', 'duration',
       'content', 'type', 'course_type', 'course_category', 'category_id',
       'featured', 'tags', 'date_created'],
      dtype='object')

In [50]:
# replacing white space in columns to NaN
df_course.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [51]:
# Remove category Sandbox and Archive. Reset Index
df_course = df_course[(df_course['category_id'] != 495) & (df_course['category_id'] != 43)]
df_course = df_course.reset_index(drop= True)

In [52]:
df_course.dtypes

course_id           int64
course_name        object
course_language    object
location           object
duration           object
content            object
type               object
course_type        object
course_category    object
category_id         int64
featured           object
tags               object
date_created       object
dtype: object

In [53]:
# change dtypes for date_created (DateTime)
df_course['date_created'] = pd.to_datetime(df_course['date_created'])

In [54]:
# add temporary column 'Today' for calcualting the age of a course
df_course['today'] = pd.Timestamp('2021-03-08')

In [55]:
# adding a column 'course_age' 
df_course['course_age'] =  (df_course['today'] - df_course['date_created']).dt.days
df_course = df_course.drop(['today', 'date_created'], axis=1)

In [56]:
# change duration to numeric
# first change U to 0
df_course['duration'] = df_course['duration'].replace(['U'],0)
# second change dtype
df_course['duration'] = pd.to_numeric(df_course['duration'])

In [57]:
# inspect categoricals columns 
print("Number of Categories in: ")
for ColName in df_course.select_dtypes(include = np.object):
    print("{} = {}".format(ColName, len(df_course[ColName].unique())))

Number of Categories in: 
course_name = 876
course_language = 7
location = 22
content = 337
type = 3
course_type = 5
course_category = 81
featured = 2
tags = 101


In [58]:
# check for NaNs
df_course.isna().sum()/df_course.shape[0]

course_id          0.000000
course_name        0.000000
course_language    0.000000
location           0.184035
duration           0.000000
content            0.067627
type               0.000000
course_type        0.186253
course_category    0.000000
category_id        0.000000
featured           0.000000
tags               0.349224
course_age         0.000000
dtype: float64

In [18]:
# cleaning location
df_course['location'].value_counts()

TUN                        207
AMD                        166
BER                        152
MBY                         62
AMD, BER, MBY, MNL, TUN     54
BER, MBY, MNL, TUN          28
BER, TUN                    16
AMD, MBY, MNL, TUN          13
MNL                         12
MBY, MNL                    10
AMD, TUN                     3
AMD, MNL, TUN                3
BER, MBY, TUN                2
AMD, MNL                     1
MBY, MNL, TUN                1
BER, MBY, MNL                1
BER, MNL, TUN                1
AMD, BER, MBY, TUN           1
BER, MBY                     1
BER, MNL                     1
AMD, MBY                     1
Name: location, dtype: int64

In [19]:
# replacing NaN's where the language is German with 'BER'
df_course.loc[(df_course.course_language == 'German') & (df_course.location.isna() == True), 'location'] = 'BER'
# replacing NaN's where the language is French with 'TUN'
df_course.loc[(df_course.course_language == 'French') & (df_course.location.isna() == True), 'location'] = 'TUN'

In [20]:
# still 157 left - remaning will be replaced with 'Unknown'
df_course['location'].isna().sum()
df_course['location'] = df_course['location'].fillna('Unknown_Location')

In [21]:
# cleaning course_type
df_course['course_type'].value_counts()

Classroom Training     280
Self-paced Learning    227
Blended Learning       122
Micro Learning         105
Name: course_type, dtype: int64

In [22]:
# replacing NaNs in 'course_type' where 'type' is 'Classrrom Training' with 'Classroom Training'
df_course.loc[(df_course.type == 'Classrrom Training') & (df_course.course_type.isna() == True), 'course_type'] = 'Classroom Training'

In [23]:
# replacing NaNs in 'course_type' where 'type' is 'E-Learning' with 'Self-paced Learning'
df_course.loc[(df_course.type == 'E-Learning') & (df_course.course_type.isna() == True), 'course_type'] = 'Self-paced Learning' 

In [24]:
# replacing NaNs in 'course_type' where 'type' is 'E-Learning' with 'Self-paced Learning'
df_course.loc[(df_course.type == 'Blended') & (df_course.course_type.isna() == True), 'course_type'] = 'Blended Learning' 

In [25]:
# filling the NaNs in column 'tags' with 'unknown_tags'
df_course['tags'] = df_course['tags'].fillna('unknown_tags')

In [26]:
# filling the NaNs in column 'content' with 'Unknown'
df_course['content'] = df_course['content'].fillna('unknown_content')

In [27]:
df_course['content'].value_counts()

File                                                                                       73
unknown_content                                                                            61
Quiz                                                                                       37
File, Seminar, URL                                                                         35
File, Seminar                                                                              23
                                                                                           ..
H5P, Quiz, Seminar                                                                          1
Choice, File, Forum                                                                         1
Assignment, Attendance, File, Forum, Page, Quiz, SCORM package, Seminar, URL                1
Assignment, Attendance, Choice, Feedback, File, Folder, Forum, Page, Quiz, Seminar, URL     1
Lesson, Page                                                

In [28]:
# saving file for analysis in Teableau
dfTableau_p1 = df_course

In [ ]:
#content_split = df_course['content'].str.split(',', expand = True).rename(columns = lambda x: "cont"+str(x+1))

In [ ]:
#content_split

In [ ]:
#conent_vals = ['Assignment', 'Attendance', 'Book', 'Certificate', 'Chat', 'Choice', 'Database', 'Feedback', 'File', 'Folder', 'Forum', 'Glossary', 'H5P', 'Label', 'Lesson', 'Page', 'Quiz', 'SCORM package', 'Seminar', 'Survey', 'Tin Can Launch Link', 'URL', 'WebEx Meeting', 'Wiki', 'Workshop']
#Course_activities = ['Assignment', 'Certificate', 'Chat', 'Choice', 'Database', 'Feedback', 'Forum', 'Lesson', 'Quiz', 'SCORM package', 'Seminar', 'Survey', 'Wiki', 'Workshop']
#Course_resources = ['Book', 'File', 'Folder', 'Glossary', 'Label', 'Page', 'URL',]
#Course_others = ['Attendance', 'HP5', 'Tin Can Launch Link', 'WebEx Meeting']
#Course_unknown = ['Unknown']


#courses_act = '|'.join(Course_activities)
#df_course[df_course['content'].str.contains(courses_act)]['content']

In [29]:
# cutting column 'content' into multiple features
df_temp = pd.concat([df_course.drop('content', 1), df_course['content'].str.get_dummies(sep=",")], 1)

In [30]:
# cutting column 'tags' into multiple features
df_course = pd.concat([df_temp.drop('tags', 1), df_temp['tags'].str.get_dummies(sep=",")], 1)

In [31]:
# cutting column 'course_language' into multiple features
df_course = pd.concat([df_course.drop('course_language', 1), df_course['course_language'].str.get_dummies(sep=",")], 1)

In [32]:
# label encoding for column featured
df_course['featured'] = df_course['featured'].map({'Yes' : 1, 'No' : 0})

In [33]:
# cutting column 'lcoation' into multiple features
# df_course = pd.concat([df_course.drop('location', 1), df_course['location'].str.get_dummies(sep=",")], 1)

In [34]:
df_course.columns = [col.replace(" ", "_").lower() for col in df_course.columns]

In [35]:
df_course.columns

Index(['course_id', 'course_name', 'location', 'duration', 'type',
       'course_type', 'course_category', 'category_id', 'featured',
       'course_age',
       ...
       'tools', 'unknown_tags', 'video', 'website', '_french', '_german',
       'english', 'french', 'german', 'unknown'],
      dtype='object', length=105)

In [36]:
# rename column 'unknown' to 'unknown_language'
df_course['unknown_language'] = df_course['unknown']
df_course = df_course.drop(['unknown'], axis=1)

In [59]:
#df_course.head(2)

In [38]:
df_course.isna().sum()

course_id           0
course_name         0
location            0
duration            0
type                0
                   ..
_german             0
english             0
french              0
german              0
unknown_language    0
Length: 105, dtype: int64

In [39]:
#load file with completion count
df_compl = pd.read_sql_query('SELECT * FROM compl_count', engine)
df_compl

,completion_count,course_id
0,985,866
1,965,386
2,882,165
3,769,189
4,684,1175
...,...,...
465,1,293
466,1,819
467,1,1161
468,1,822


In [40]:
# creating a data set with completion count to analyis in Tableau
dfTableau = pd.merge(df_compl, dfTableau_p1, on="course_id", how='outer')
dfTableau['completion_count'] = dfTableau['completion_count'].fillna(0)
# chage type float to int
dfTableau['completion_count'] = dfTableau['completion_count'].astype(np.int64)
dfTableau.to_csv(r'/users/mzimmermann/ironhack/CLZ/Tableau/dfTableau.csv', index = False)

In [ ]:
# merge dataframes with OUTER join -> for no values it will give a NaN
df_merged = pd.merge(df_compl, df_course, on="course_id", how='outer')

In [ ]:
df_course[df_course['course_id'] == 814]

In [ ]:
# replace NaN with 0
df_merged['completion_count'] = df_merged['completion_count'].fillna(0)
# chage type float to int
df_merged['completion_count'] = df_merged['completion_count'].astype(np.int64)

In [ ]:
# drop irrelevant columns
data = df_merged.drop(['location','type', 'category_id', 'course_name'], axis=1)
data.head(1)

In [ ]:
data.shape

In [ ]:
data.to_csv(r'/users/mzimmermann/ironhack/CLZ/cleaned_files/data.csv', index = False)

In [ ]:
# loading file where courses are reviewed
df_reviewed = pd.read_excel('CLZ_BER_Courses_OS.xlsx').drop('course_name', axis=1)

In [ ]:
# merge data with df_reviewed
data_revd = pd.merge(data, df_reviewed, on="course_id")
data_revd

In [ ]:
data_revd.to_csv(r'/users/mzimmermann/ironhack/CLZ/cleaned_files/data_revd.csv', index = False)

In [ ]:
data_revd['keep'].value_counts()